![Renode](https://antmicro.com/OpenSource/assets/images/projects/renode.png)

## Install requirements

In [ ]:
!pip install -q git+https://github.com/antmicro/pyrenode.git git+https://github.com/antmicro/renode-colab-tools.git
!mkdir -p renode && cd renode && wget https://dl.antmicro.com/projects/renode/builds/renode-latest.linux-portable.tar.gz && tar -xzf renode-latest.linux-portable.tar.gz --strip 1
!pip install -q -r renode/tests/requirements.txt
!git clone --quiet --recurse-submodules https://github.com/antmicro/tensorflow-zephyr-vexriscv-examples.git

import os
from renode_colab_tools import metrics
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']
os.environ['TENSORFLOW_PATH'] = os.getcwd()+"/tensorflow-zephyr-vexriscv-examples/tensorflow"
os.environ['ZEPHYR_SDK_INSTALL_DIR'] = "/opt/zephyr-sdk"
os.environ['ZEPHYR_TOOLCHAIN_VARIANT'] = "zephyr"

In [ ]:
sudo apt update && sudo apt install -y cmake ninja-build gperf ccache dfu-util device-tree-compiler wget python python3-pip python3-setuptools python3-tk python3-wheel xz-utils file make gcc gcc-multilib locales tar curl unzip xxd
pip install --upgrade cmake virtualenv
wget https://github.com/zephyrproject-rtos/sdk-ng/releases/download/v0.11.2/zephyr-sdk-0.11.2-setup.run
chmod +x zephyr-sdk-0.11.2-setup.run
./zephyr-sdk-0.11.2-setup.run -- -y -d $ZEPHYR_SDK_INSTALL_DIR
sudo apt install patchelf -y
sudo patchelf --set-interpreter /lib64/ld-linux-x86-64.so.2 $ZEPHYR_SDK_INSTALL_DIR/sysroots/x86_64-pokysdk-linux/usr/bin/dtccd $TENSORFLOW_PATH && make -f tensorflow/lite/micro/tools/make/Makefile TARGET=zephyr_vexriscv hello_world_bin
mkdir -p binaries/hello_world && cp $TENSORFLOW_PATH/tensorflow/lite/micro/tools/make/gen/zephyr_vexriscv_x86_64_default/hello_world/build/zephyr/zephyr.elf binaries/hello_world/hello_world_zephyr.elf

## Run the hello-world example in Renode

In [ ]:
import time
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)
tell_renode('using sysbus')
tell_renode('mach create')
tell_renode('machine LoadPlatformDescription @tensorflow-zephyr-vexriscv-examples/examples/litex-vexriscv-tflite.repl')
tell_renode('sysbus LoadELF @binaries/hello_world/hello_world_zephyr.elf')

tell_renode('uart CreateFileBackend @uart.dump true')
tell_renode('logLevel 3')
tell_renode('machine EnableProfiler @metrics.dump')
tell_renode('s')
while not os.path.exists('renode/uart.dump'):
  time.sleep(1) #waits for creating uart.dump
!timeout 60 tail -c+2 -f renode/uart.dump | sed '/^1$/ q'
tell_renode('q')
expect_cli('Renode is quitting')
time.sleep(1) #wait not to kill Renode forcefully
shutdown_renode()

## Renode metrics analysis

In [ ]:
from renode.tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('renode/metrics.dump')

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_executed_instructions(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_memory_access(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_exceptions(parser)

In [ ]:
metrics.configure_plotly_browser_state()
metrics.show_peripheral_access(parser)